In [9]:
!pip install torch torchvision matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 3.2 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 4.9 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]


In [2]:
import torch
import os
import utils
from datetime import datetime
from data_roboflow import YoloRoboflowDataset
from models import YOLOv1
from torch.utils.data import DataLoader

# Configuration parameters
model_dir = None  # Directory containing model weights
weights = 'final'  # Name of weights file to use (without extension)
batch_size = 8  # Batch size for evaluation
iou_threshold = 0.5  # IoU threshold for mAP calculation
confidence_threshold = 0.1  # Confidence threshold for detections

# Set model directory if not provided
if model_dir is None:
    # Find most recent model directory
    base_dir = 'models/yolo_v1'
    if os.path.exists(base_dir):
        dates = os.listdir(base_dir)
        if dates:
            latest_date = max(dates)
            times = os.listdir(os.path.join(base_dir, latest_date))
            if times:
                latest_time = max(times)
                model_dir = os.path.join(base_dir, latest_date, latest_time)
                print(f"Using most recent model directory: {model_dir}")
# Assert that model directory exists
assert model_dir is not None and os.path.exists(model_dir), "No model directory found. Please specify model_dir manually"

# Load class names
classes = utils.load_class_array()
print(f"Loaded {len(classes)} classes: {classes}")

# Create dataset and dataloader
print("Loading validation dataset...")
dataset = YoloRoboflowDataset('valid', normalize=True, augment=False)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Load model
print(f"Loading model from {model_dir}...")
model = YOLOv1()
model.eval()
weights_path = os.path.join(model_dir, 'weights', weights)
model.load_state_dict(torch.load(weights_path, map_location=utils.device))
model = model.to(utils.device)

Using most recent model directory: models/yolo_v1/04_29_2025/13_11_17
Loaded 2 classes: ['pig', 'worker']
Loading validation dataset...
Loading model from models/yolo_v1/04_29_2025/13_11_17...


RuntimeError: Error(s) in loading state_dict for YOLOv1:
	Missing key(s) in state_dict: "model.17.weight", "model.17.bias", "model.23.weight", "model.23.bias", "model.29.weight", "model.29.bias", "model.33.weight", "model.33.bias", "model.36.weight", "model.36.bias", "model.40.weight", "model.40.bias", "model.42.weight", "model.42.bias", "model.44.weight", "model.44.bias", "model.48.weight", "model.48.bias", "model.51.weight", "model.51.bias", "model.54.weight", "model.54.bias". 
	Unexpected key(s) in state_dict: "model.16.weight", "model.16.bias", "model.18.weight", "model.18.bias", "model.22.weight", "model.22.bias", "model.24.weight", "model.24.bias", "model.28.weight", "model.28.bias", "model.32.weight", "model.32.bias", "model.34.weight", "model.34.bias", "model.35.weight", "model.35.bias", "model.41.weight", "model.41.bias", "model.43.weight", "model.43.bias", "model.49.weight", "model.49.bias". 
	size mismatch for model.19.weight: copying a param with shape torch.Size([512, 256,

In [1]:
import torch
import matplotlib.pyplot as plt
import supervision as sv
import cv2
from tqdm import tqdm
import numpy as np

# Get a single image
random_idx = torch.randint(0, len(dataset), (1,)).item()
image, label, original = dataset[random_idx]

# Add batch dimension
image = image.unsqueeze(0).to(utils.device)

# Get predictions
model.eval()
with torch.no_grad():
    predictions = model(image)

# Convert original image to numpy array for supervision visualization
original_np = original.permute(1, 2, 0).numpy()
original_np = (original_np * 255).astype('uint8')

# Get bounding boxes from predictions
pred_boxes = utils.extract_bboxes_from_yolo_output(predictions[0], min_confidence=0.1)
pred_boxes = utils.non_max_suppression(pred_boxes, iou_threshold=0.5)

pred_boxes


NameError: name 'dataset' is not defined

In [4]:
# Create results directory if needed
results_dir = os.path.join('results', 'evaluation', datetime.now().strftime('%Y%m%d_%H%M%S'))
os.makedirs(results_dir, exist_ok=True)

# Evaluate model
print(f"Evaluating model on {len(dataset)} validation images...")
results = utils.evaluate_model(model, loader, classes)

# Print results
print("\nEvaluation Results:")
print("-" * 50)

# Print mAP metrics
mAP_metrics = [key for key in results.keys() if key.startswith('mAP')]
for metric in mAP_metrics:
    print(f"{metric}: {results[metric]:.4f}")

print("\nPer-class Average Precision:")
print("-" * 50)

# Print AP metrics for each class
for class_idx, class_name in enumerate(classes):
    ap_05 = results.get(f"AP@0.5_{class_name}", 0)
    ap_075 = results.get(f"AP@0.75_{class_name}", 0)
    print(f"{class_name}: AP@0.5={ap_05:.4f}, AP@0.75={ap_075:.4f}")

NameError: name 'os' is not defined